In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [17]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load data
df = pd.read_csv("Results_21Mar2022.csv")

# Columns to use
env_columns = [
    'mean_ghgs', 'mean_land', 'mean_watscar',
    'mean_eut', 'mean_ghgs_ch4', 'mean_ghgs_n2o',
    'mean_bio', 'mean_watuse', 'mean_acid'
]

# Normalize for plotting
def normalize_df(data):
    norm = data.copy()
    for col in env_columns:
        norm[col] = (norm[col] - norm[col].min()) / (norm[col].max() - norm[col].min())
    return norm

# Start Dash app
app = dash.Dash(__name__)
server = app.server  # for deployment

app.layout = html.Div([
    html.H2(
    "Environmental Impact of Diets by Group"),
    html.Div([
        html.Label("Select Diet Group:"),
        dcc.Dropdown(
            options=[{"label": d, "value": d} for d in sorted(df["diet_group"].unique())],
            value=None,
            id='diet-dropdown',
            multi=True
        ),
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),

    html.Div([
        html.Label("Select Gender:"),
        dcc.Dropdown(
            options=[{"label": s, "value": s} for s in sorted(df["sex"].unique())],
            value=None,
            id='gender-dropdown',
            multi=True
        ),
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),

    html.Div([
        html.Label("Select Age Group:"),
        dcc.Dropdown(
            options=[{"label": a, "value": a} for a in sorted(df["age_group"].unique())],
            value=None,
            id='age-dropdown',
            multi=True
        ),
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),

    dcc.Graph(id="parallel-plot")
])

@app.callback(
    Output("parallel-plot", "figure"),
    Input("diet-dropdown", "value"),
    Input("gender-dropdown", "value"),
    Input("age-dropdown", "value"),
)
def update_plot(selected_diets, selected_gender, selected_age):
    filtered_df = df.copy()

    if selected_diets:
        filtered_df = filtered_df[filtered_df["diet_group"].isin(selected_diets)]
    if selected_gender:
        filtered_df = filtered_df[filtered_df["sex"].isin(selected_gender)]
    if selected_age:
        filtered_df = filtered_df[filtered_df["age_group"].isin(selected_age)]

    # Normalize and prepare plot
    norm_df = normalize_df(filtered_df)
    if norm_df.empty:
        return px.parallel_coordinates(pd.DataFrame(columns=env_columns), dimensions=env_columns)

    fig = px.parallel_coordinates(
        norm_df,
        dimensions=env_columns,
        color=filtered_df["diet_group"].astype('category').cat.codes,
        color_continuous_scale='Jet',
        labels={col: col.replace("mean_", "").upper() for col in env_columns}
    )
    fig.update_layout(
        title="Environmental Impacts across Dietary Metrics",
        margin=dict(t=80, b=10),
        coloraxis_colorbar=dict(title="Diet Code"),
        plot_bgcolor='white'
    )
    return fig

if __name__ == "__main__":
    app.run(debug=True)

<IPython.core.display.Javascript object>